# Projects for Analysis of Anti-Depressant Drugs’s Adverse Events by FDA Adverse Event Reporting System (FAERS) from January 2019 to December 2023

## 1. MERGE DATA

In [10]:
# load library
import pandas as pd
import numpy as np
import re

#### 1.1 Function to merge data

In [2]:
# FUNCTION OF MERGING DATA

import re
import pandas as pd

def process_faers_data():
    
    # Define normalise text
    def normalize_text(text):
        """Normalize text: remove punctuation and spaces, convert to lowercase."""
        if pd.isnull(text):
            return ''
        return re.sub(r'[^\w\s]', '', text).replace('  ', ' ').lower()

    # Define keywords for filtering indications
    keywords = ['depres', 'sad', 'mdd', 'dysthymia']
    pattern_strings = [re.escape(keyword) for keyword in keywords]
    keyword_pattern = re.compile('|'.join(pattern_strings), re.IGNORECASE)

    filename_prefixes = ['DEMO', 'DRUG', 'INDI', 'OUTC', 'REAC', 'RPSR', 'THER']
    final_faers = pd.DataFrame()  # Start with an empty DataFrame

    # Iterate over years and quarters in reverse order (most recent first)
    for year in reversed(range(18, 24)):  # Years 2023 to 2018
        for quarter in reversed(range(1, 5)):  # Quarters 4 to 1
            dict_df = {}
            try:
                # Load and merge all relevant files for the current quarter
                for prefix in filename_prefixes:
                    filename = f'{prefix}{year}Q{quarter}'
                    print(f"Processing {filename}")
                    dict_df[prefix] = pd.read_csv(
                        f'data/{filename}.txt', sep='$', encoding_errors='ignore'
                    )

                # Use DELETE18Q4 for all quarters in 2018, or the regular DELETE file for other years
                deleted_case_filename = 'data/DELETE18Q4.txt' if year == 18 else f'data/DELETE{year}Q{quarter}.txt'
                deleted = pd.read_csv(
                    deleted_case_filename,
                    sep='$',
                    encoding_errors='ignore',
                    names=['caseid']
                )

                # Normalize 'indi_pt' in 'INDI' dataframe
                dict_df['INDI']['indi_pt'] = dict_df['INDI']['indi_pt'].apply(normalize_text)

                # Filter 'INDI' for relevant indications
                dict_df['INDI'] = dict_df['INDI'][
                    dict_df['INDI']['indi_pt'].str.contains(keyword_pattern, na=False)
                ]

                # Merge data incrementally
                df = pd.merge(
                    dict_df['DRUG'], dict_df['INDI'],
                    left_on=['primaryid', 'drug_seq'],
                    right_on=['primaryid', 'indi_drug_seq'],
                    suffixes=(None, '_DROP')
                )
                df = pd.merge(df, dict_df['THER'], how='left',
                              left_on=['primaryid', 'drug_seq'],
                              right_on=['primaryid', 'dsg_drug_seq'], suffixes=(None, '_DROP'))
                df = pd.merge(df, dict_df['DEMO'], on='primaryid', suffixes=(None, '_DROP'))
                df = pd.merge(df, dict_df['REAC'], how='left', on='primaryid', suffixes=(None, '_DROP'))
                df = pd.merge(df, dict_df['OUTC'], how='left', on='primaryid', suffixes=(None, '_DROP'))
                df = pd.merge(df, dict_df['RPSR'], how='left', on='primaryid', suffixes=(None, '_DROP'))

                # Remove columns with '_DROP' suffix
                df = df.loc[:, ~df.columns.str.endswith('_DROP')]

                # Remove records with deleted cases
                df = df[~df['caseid'].isin(deleted['caseid'])]

                # Convert 'caseversion' to numeric for sorting
                df['caseversion'] = pd.to_numeric(df['caseversion'], errors='coerce')

                # Sort by 'caseid' and 'caseversion' (latest first)
                df = df.sort_values(by=['caseid', 'caseversion'], ascending=[True, False])

                # Keep all rows for the latest caseversion of each case
                latest_versions = df.drop_duplicates(subset=['caseid'], keep='first')
                df = df[df['caseid'].isin(latest_versions['caseid']) & df['caseversion'].isin(latest_versions['caseversion'])]

                # Remove exact duplicates across all columns
                df = df.drop_duplicates()

                # Append to final dataframe
                if not final_faers.empty:
                    final_faers = pd.concat([final_faers, df])
                else:
                    final_faers = df

                print(f"Finished processing {filename}. Records: {len(df)}")
            except FileNotFoundError:
                print(f"File not found for {year}Q{quarter}. Skipping.")
            except Exception as e:
                print(f"An error occurred while processing {year}Q{quarter}: {e}")

    # Final deduplication for the concatenated dataset
    final_faers = final_faers.drop_duplicates()

    print(f"Total records after merging: {len(final_faers)}")
    print(f"Number of unique cases: {final_faers['caseid'].nunique()}")

    # Optionally, save the final dataframe to a CSV file
    final_faers.to_csv('FAERS_merged_data.csv', index=False)

    return final_faers

In [3]:
# run function to merge the data & save as 'FAERS_merged_data.csv' - remove '#' below if want to re-run
# merged_data = process_faers_data()

Processing DEMO23Q4


/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/1097199328.py:32: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_df[prefix] = pd.read_csv(


Processing DRUG23Q4
Processing INDI23Q4
Processing OUTC23Q4
Processing REAC23Q4
Processing RPSR23Q4
Processing THER23Q4
Finished processing THER23Q4. Records: 59726
Processing DEMO23Q3


/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/1097199328.py:32: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_df[prefix] = pd.read_csv(


Processing DRUG23Q3
Processing INDI23Q3
Processing OUTC23Q3
Processing REAC23Q3
Processing RPSR23Q3
Processing THER23Q3
Finished processing THER23Q3. Records: 71450
Processing DEMO23Q2


...

Processing DRUG18Q1


/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/1097199328.py:32: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_df[prefix] = pd.read_csv(


Processing INDI18Q1
Processing OUTC18Q1
Processing REAC18Q1
Processing RPSR18Q1
Processing THER18Q1


/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/1097199328.py:32: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_df[prefix] = pd.read_csv(


Finished processing THER18Q1. Records: 67335
Total records after merging: 1904757
Number of unique cases: 112785
Is 'caseid' unique? False


==> The merged data is already removed identical duplication and keep the duplicated primaryid with the latest case version.

#### 1.2 Overview initial dataset

In [9]:
merged_data = pd.read_csv('FAERS_merged_data.csv')

merged_data.info()

/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/81723904.py:1: DtypeWarning: Columns (12,26,37,46,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv('FAERS_merged_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904757 entries, 0 to 1904756
Data columns (total 54 columns):
 #   Column            Dtype  
---  ------            -----  
 0   primaryid         int64  
 1   caseid            int64  
 2   drug_seq          int64  
 3   role_cod          object 
 4   drugname          object 
 5   prod_ai           object 
 6   val_vbm           int64  
 7   route             object 
 8   dose_vbm          object 
 9   cum_dose_chr      float64
 10  cum_dose_unit     object 
 11  dechal            object 
 12  rechal            object 
 13  lot_num           object 
 14  exp_dt            float64
 15  nda_num           float64
 16  dose_amt          float64
 17  dose_unit         object 
 18  dose_form         object 
 19  dose_freq         object 
 20  indi_drug_seq     int64  
 21  indi_pt           object 
 22  dsg_drug_seq      float64
 23  start_dt          float64
 24  end_dt            float64
 25  dur               float64
 26  dur_cod       

In [11]:
merged_data.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
primaryid,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821,675390821
caseid,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908,6753908
drug_seq,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
role_cod,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I,I
drugname,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS,ISENTRESS
prod_ai,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM,RALTEGRAVIR POTASSIUM
val_vbm,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
route,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral,Oral
dose_vbm,"400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid","400 milligram, 4 every 1 days/qid"
cum_dose_chr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
merged_data['indi_pt'].value_counts()

depression                                                   1543707
major depression                                              167668
antidepressant therapy                                         79963
depressed mood                                                 30017
schizoaffective disorder depressive type                       19223
mixed anxiety and depressive disorder                          19094
depressive symptom                                             14473
persistent depressive disorder                                  8733
adjustment disorder with depressed mood                         5693
perinatal depression                                            3904
depression suicidal                                             3852
torsade de pointes                                              2503
depressed level of consciousness                                1399
respiratory depression                                           897
agitated depression               

In [13]:
# # Group the data by 'primaryid' and aggregate the indications
# aggregated_data = merged_data.groupby('primaryid')['indi_pt'].apply(lambda x: ', '.join(x.unique())).reset_index()

# # Count the frequency of each unique indication across all primaryids
# value_counts = aggregated_data['indi_pt'].value_counts()

# # Create a formatted DataFrame for LaTeX table
# formatted_table = pd.DataFrame({
#     'Initial Indication': value_counts.index,
#     'Frequency': value_counts.values
# })

# # Specify the column format for the LaTeX table
# column_format = 'l@{}r'

# # Generate the LaTeX table
# latex_table = formatted_table.to_latex(
#     index=False, 
#     header=True, 
#     column_format=column_format,
#     escape=False
# )

# # Save the LaTeX table to a file
# with open('Initial_Indication.tex', 'w') as f:
#     f.write(latex_table)

# # Optionally, print the first few rows of the formatted table
# print(formatted_table.head())


                      Initial Indication  Frequency
0                             depression     107632
1                       major depression      12794
2                 antidepressant therapy       4725
3                         depressed mood       2034
4  mixed anxiety and depressive disorder       1898


/var/folders/_z/wx40qqws337c64bjr5v3vh_80000gn/T/ipykernel_40954/238896575.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = formatted_table.to_latex(


> Refer fda_step2_clean for next part of cleaning.